In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [4]:
course_list = ['청도','포항','중문','시흥','거창','속리산','춘천']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [5]:
AWS_folder = os.getcwd()

b_folder = os.path.join(AWS_folder,'BACKUPS')
b_folder

src_folder = "D:\ToAWS2\ds_labelinfo"
src_folder

out_folder = os.path.join(AWS_folder,'filepathBackUp')
out_folder

'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp'

In [6]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError



recentDB = glob.glob(os.path.join(b_folder ,'Total*.pickle'), recursive=True)[-1]
df = pd.read_pickle(recentDB)

In [7]:
df.shape

(184082, 7)

In [8]:
maxLabel = glob.glob(os.path.join(src_folder,'**/MGC*.json'), recursive=True)

In [9]:
oldLabels = []

for dataJson_ in maxLabel:
  with open(dataJson_, "r", encoding='utf-8') as _json:
    oldLabels.extend(json.load(_json))


In [10]:
len(oldLabels)

48587

In [11]:
oldLabels[1]

{'id': '20230811142508_1293396509722_362175400556_117497000',
 '11originalFileJPG': 'ta/nas/11_원천데이터/G05_포항CC/CU_근접촬영/05_병해잔디/84_(토양병)페어리링/G05_CU_0584_DJI_20230811142508_0023_D.JPG',
 '11originalFileGrp': ['ta/nas/11_원천데이터/G05_포항CC/CU_근접촬영/05_병해잔디/84_(토양병)페어리링/G05_CU_0584_DJI_20230811142508_0023_MS_G.TIF',
  'ta/nas/11_원천데이터/G05_포항CC/CU_근접촬영/05_병해잔디/84_(토양병)페어리링/G05_CU_0584_DJI_20230811142508_0023_MS_NIR.TIF',
  'ta/nas/11_원천데이터/G05_포항CC/CU_근접촬영/05_병해잔디/84_(토양병)페어리링/G05_CU_0584_DJI_20230811142508_0023_MS_R.TIF',
  'ta/nas/11_원천데이터/G05_포항CC/CU_근접촬영/05_병해잔디/84_(토양병)페어리링/G05_CU_0584_DJI_20230811142508_0023_MS_RE.TIF'],
 'GeoTagInfo': {'coords': [129.33965097222224, 36.21754005555556],
  'datetime_original': '2023:08:11 14:25:08',
  'gps_altitude': 117.497,
  'gps_altitude_ref': 0},
 'label': {'level1': '05_병해잔디',
  'level2': '84_(토양병)페어리링',
  'level3': '',
  'TurfType': ''}}

In [12]:
maxdf = pd.DataFrame(oldLabels)
maxdf.head(2)

,id,11originalFileJPG,11originalFileGrp,GeoTagInfo,label
0,20230811155905_1293381201667_362176992778_1471...,ta/nas/11_원천데이터/G05_포항CC/CU_근접촬영/05_병해잔디/84_(토...,[ta/nas/11_원천데이터/G05_포항CC/CU_근접촬영/05_병해잔디/84_(...,"{'coords': [129.33812016666667, 36.21769927777...","{'level1': '05_병해잔디', 'level2': '84_(토양병)페어리링'..."
1,20230811142508_1293396509722_362175400556_1174...,ta/nas/11_원천데이터/G05_포항CC/CU_근접촬영/05_병해잔디/84_(토...,[ta/nas/11_원천데이터/G05_포항CC/CU_근접촬영/05_병해잔디/84_(...,"{'coords': [129.33965097222224, 36.21754005555...","{'level1': '05_병해잔디', 'level2': '84_(토양병)페어리링'..."


In [13]:
maxdf['oldlabelL1'] = maxdf['label'].map(lambda x: x.get('level1',np.nan))
maxdf['oldlabelL2'] = maxdf['label'].map(lambda x: x.get('level2',np.nan))
maxdf['oldlabelL3'] = maxdf['label'].map(lambda x: x.get('level3',np.nan))
maxdf['oldlabelType'] = maxdf['label'].map(lambda x: x.get('TurfType',np.nan))

maxdf = maxdf.replace('', np.nan, regex=True)

In [14]:
maxdf['oldlabelL1'].unique(), maxdf['oldlabelL2'].unique(), maxdf['oldlabelL3'].unique(), maxdf['oldlabelType'].unique()

(array(['05_병해잔디', '02_물리적손상잔디', '01_건강한잔디', '04_잡초', '03_충해잔디', '31_기타충해',
        '01_일반', '18_건조피해', '21_디봇'], dtype=object),
 array(['84_(토양병)페어리링', '79_(엽고성병)피티움블라이트', '77_(엽고성병)엽고병', '87_기타병해',
        '86_(조류)남조류', '76_(엽고성병)탄저병', '83_(패취류)섬머패취', '81_패취류', '17_볼마크',
        '19_플레이어답압', '21_디봇', '18_건조피해', '20_고온피해', '25_장비피해', '22_과습피해',
        '27_기타피해', '27_기타물리피해', '01_일반', '16_기타', '71_(이종잔디)기타',
        '56_(광엽)민들레', '72_(이종잔디)이종잔디(벤트)', '73_(이종잔디)이종잔디(켄터키)',
        '54_(광엽)매듭풀', '37_(화본과)새포아풀', '36_(화본과)바랭이', '32_화본과', '40_사초과',
        '28_나방류', '85_(토양병)루트피티움', '02_배토', '78_(엽고성병)동전마름병',
        '38_(화본과)왕바랭이', '63_(광엽)애기땅빈대', '43_(광엽)개갓냉이', '47_(광엽)기타',
        '74_(이끼)은이끼', '69_(광엽)토끼풀', '34_(화본과)기타', '53_(광엽)망초',
        '41_(사초과)방동사니', '46_(광엽)괭이밥', '58_(광엽)벼룩이자리', '62_(광엽)쇠비름',
        '68_(광엽)질경이', '66_(광엽)제비꽃', '35_(화본과)띠', '32_(화본과)강아지풀',
        '44_(광엽)개망초', '42_(사초과)파대가리', '75_(기타잡초)기타잡초', '61_(광엽)선피막이',
        '70_(광엽)피막이', '55_(광엽)명아주', '67_(광엽)중대가리풀', '

In [15]:
L1MAP = {
  '05_병해잔디':'병해',
  '02_물리적손상잔디':'물리피해',
  '01_건강한잔디':'건강한잔디',
  '04_잡초':'잡초',
  '03_충해잔디':'충해',
  '31_기타충해':'충해',
  '01_일반':'건강한잔디',
  '18_건조피해':'물리피해',
  '21_디봇':'물리피해'
}

L2MAP = {
'토양병':'토양병',
'엽고성병':'엽고성',
'조류':'조류',
'패취류':'패취',
'이종잔디':'이종잔디',
'광엽':'광엽',
'화본과':'화본과',
'이끼':'이끼',
'사초과':'사초과',
'기타잡초':'기타잡초',
'87_기타병해':'기타병해',
'81_패취류':'패취',
'17_볼마크':'볼마크',
'19_플레이어답압':'기타물리',
'21_디봇':'디봇',
'18_건조피해':'기타물리',
'20_고온피해':'기타물리',
'25_장비피해':'장비피해',
'22_과습피해':'기타물리',
'27_기타피해':'기타물리',
'27_기타물리피해':'기타물리',
'01_일반':'일반',
'16_기타':'일반',
'32_화본과':'화본과',
'40_사초과':'사초과',
'28_나방류':'기타물리',
'02_배토':'일반',
'43_광엽':'광엽',
'74_이끼':'이끼',
'76_엽고성병':'엽고성',
'24_음지피해':'기타물리',
'07_깎기':'일반',
'08_보식':'일반',
'31_기타충해':'기타충해',
'30_응애':'기타물리',
'G01_CU_0331_NIR없은이미지':'NIRERROR',
'NIR없는이미지_G01_CU_0101':'NIRERROR',
'G01_CU_0218_NIR없는이미지_208건':'NIRERROR',
'G01_CU_0221_ NIR파일없는이미지_740건':'NIRERROR'
}

In [16]:
maxdf['oldlabelL1'] = maxdf['oldlabelL1'].map(L1MAP)

In [17]:
def findL3(x):
  t_ = x.split(')') if ')' in x else None
  if t_:
    ans = t_[-1]
  else:
    ans = np.nan
  return ans


maxdf['oldlabelL3'] = maxdf['oldlabelL2'].map(lambda x: findL3(x))

In [18]:
def findL2(x):
  if '(' not in x:
    return x
  t_ = re.search(r"\((\w+)\)", x)
  if t_:
    ans = t_.groups()[0]
  else:
    ans = np.nan
  return ans


maxdf['oldlabelL2'] = maxdf['oldlabelL2'].map(lambda x: findL2(x))
maxdf['oldlabelL2'] = maxdf['oldlabelL2'].map(L2MAP)

In [19]:
maxdf['oldlabelL1'].unique(), maxdf['oldlabelL2'].unique(), maxdf['oldlabelL3'].unique(), maxdf['oldlabelType'].unique()

(array(['병해', '물리피해', '건강한잔디', '잡초', '충해'], dtype=object),
 array(['토양병', '엽고성', '기타병해', '조류', '패취', '볼마크', '기타물리', '디봇', '장비피해',
        '일반', '이종잔디', '광엽', '화본과', '사초과', '이끼', '기타잡초', '기타충해', 'NIRERROR'],
       dtype=object),
 array(['페어리링', '피티움블라이트', '엽고병', nan, '남조류', '탄저병', '섬머패취', '기타', '민들레',
        '', '매듭풀', '새포아풀', '바랭이', '루트피티움', '동전마름병', '왕바랭이', '애기땅빈대',
        '개갓냉이', '은이끼', '토끼풀', '망초', '방동사니', '괭이밥', '벼룩이자리', '쇠비름', '질경이',
        '제비꽃', '띠', '강아지풀', '개망초', '파대가리', '기타잡초', '선피막이', '피막이', '명아주',
        '중대가리풀', '브라운패취', '라지패취', '마디풀'], dtype=object),
 array([nan]))

In [20]:
maxdf[maxdf['oldlabelL2'].isna()]

,id,11originalFileJPG,11originalFileGrp,GeoTagInfo,label,oldlabelL1,oldlabelL2,oldlabelL3,oldlabelType


In [21]:
maxdf = maxdf.fillna('')

In [22]:
def createNewLabel(L1,L2,L3,Type):
  return {
    'level1':L1,
    'level2':L2,
    'level3':L3,
    'TurfType':Type
  }

In [23]:
maxdf['newLabel'] = maxdf.apply(lambda x : createNewLabel(x['oldlabelL1'],x['oldlabelL2'],x['oldlabelL3'],x['oldlabelType']), axis=1)

In [24]:
maxdf = maxdf[['id','label','newLabel']]

In [25]:
out_folder = r"D:\NewDB"

In [26]:
maxdf.groupby(['id']).agg({'label':list,'newLabel':list}).reset_index().to_excel(os.path.join(out_folder,'MAXLABEL_{}.xlsx'.format(datetime.now().strftime("%Y%m%d_%H%M%S"))))

In [27]:
maxdf = maxdf.groupby(['id']).agg({'label':list,'newLabel':list}).reset_index()

In [28]:
maxdf.iloc[0,:].newLabel

[{'level1': '병해', 'level2': '토양병', 'level3': '페어리링', 'TurfType': ''}]

In [29]:
def addTBD(x):
  x.append({'level1': 'TBD', 'level2': '', 'level3': '', 'TurfType': ''})
  return x
maxdf.newLabel = maxdf.newLabel.map(lambda x: addTBD(x))

In [30]:
maxdf.iloc[0,:].newLabel

[{'level1': '병해', 'level2': '토양병', 'level3': '페어리링', 'TurfType': ''},
 {'level1': 'TBD', 'level2': '', 'level3': '', 'TurfType': ''}]

In [31]:
sumdf = df.groupby(['id']).agg({'id':'count','metadata':list}).rename(columns = {'id':'idcnt'}).reset_index()

In [32]:
sumdf[sumdf.idcnt>1].to_excel(os.path.join(out_folder,'DuplicatedFile{}.xlsx'.format(datetime.now().strftime("%Y%m%d_%H%M%S"))))

In [33]:
newdf = df.merge(maxdf[['id', 'newLabel']], on = ['id'], how = 'left')
df.shape, newdf.shape

((184082, 7), (184082, 8))

In [34]:
newdf.head(2)

,metadata,date,area,courseid,json,id,desc,newLabel
0,"MGC00320230910해안 1H물리적피해,디봇,충해,땅강아지,기타,버뮤다그래스,...",20230910,해안 1H,MGC003,{'originalFileGrp': ['ta/nas/10_원시데이터(3사_분류대상)...,20230910155330_1264016413333_332480451667_8602...,"물리적피해,디봇,충해,땅강아지,기타,버뮤다그래스,페어웨이,러프,화본과,잡초,띠_3",NaN
1,"MGC00320230910해안 1H물리적피해,디봇,충해,땅강아지,기타,버뮤다그래스,...",20230910,해안 1H,MGC003,{'originalFileGrp': ['ta/nas/10_원시데이터(3사_분류대상)...,20230910155332_1264016348611_332480353889_8615...,"물리적피해,디봇,충해,땅강아지,기타,버뮤다그래스,페어웨이,러프,화본과,잡초,띠_3",NaN


In [35]:
newdf = newdf[~newdf.newLabel.isna()]
newdf['numLabel'] = newdf.json.map(lambda x: len(x.get('label',[])))


In [36]:
newdf

,metadata,date,area,courseid,json,id,desc,newLabel,numLabel
2465,MGC00520230817거창 EAST_1H미기재_1,20230817,거창 EAST_1H,MGC005,{'originalFileGrp': ['NIA_2023_52_잔디\10_원시데이터(...,20230817141639_1279075913333_355935636389_6999...,미기재_1,"[{'level1': '물리피해', 'level2': '디봇', 'level3': ...",0
2466,MGC00520230817거창 EAST_1H미기재_1,20230817,거창 EAST_1H,MGC005,{'originalFileGrp': ['NIA_2023_52_잔디\10_원시데이터(...,20230817141641_1279075915000_355935639722_6999...,미기재_1,"[{'level1': '물리피해', 'level2': '디봇', 'level3': ...",0
2467,MGC00520230817거창 EAST_1H미기재_1,20230817,거창 EAST_1H,MGC005,{'originalFileGrp': ['NIA_2023_52_잔디\10_원시데이터(...,20230817141643_1279075916111_355935647222_6998...,미기재_1,"[{'level1': '물리피해', 'level2': '디봇', 'level3': ...",0
2468,MGC00520230817거창 EAST_1H미기재_1,20230817,거창 EAST_1H,MGC005,{'originalFileGrp': ['NIA_2023_52_잔디\10_원시데이터(...,20230817141645_1279076533056_355936205000_6997...,미기재_1,"[{'level1': '물리피해', 'level2': '디봇', 'level3': ...",0
2469,MGC00520230817거창 EAST_1H미기재_1,20230817,거창 EAST_1H,MGC005,{'originalFileGrp': ['NIA_2023_52_잔디\10_원시데이터(...,20230817141647_1279077807778_355937316944_6998...,미기재_1,"[{'level1': '물리피해', 'level2': '디봇', 'level3': ...",0
...,...,...,...,...,...,...,...,...,...
182932,"MGC00420230822아세코벨리 1H칼라켄터키블루그래스지역,패취병,옐로우패취,이...",20230822,아세코벨리 1H,MGC004,{'originalFileGrp': ['NIA_2023_52_잔디\10_원시데이터(...,20230822133938_1267709378889_373703389722_6686...,"칼라켄터키블루그래스지역,패취병,옐로우패취,이종잔디,로이시아,크리핑벤트,화본과,바랭이_0","[{'level1': '잡초', 'level2': '화본과', 'level3': '...",0
182937,"MGC00420230822아세코벨리 1H칼라켄터키블루그래스지역,패취병,옐로우패취,이...",20230822,아세코벨리 1H,MGC004,{'originalFileGrp': ['NIA_2023_52_잔디\10_원시데이터(...,20230822133948_1267709457222_373703283333_6688...,"칼라켄터키블루그래스지역,패취병,옐로우패취,이종잔디,로이시아,크리핑벤트,화본과,바랭이_0","[{'level1': '잡초', 'level2': '화본과', 'level3': '...",0
182940,"MGC00420230822아세코벨리 1H칼라켄터키블루그래스지역,패취병,옐로우패취,이...",20230822,아세코벨리 1H,MGC004,{'originalFileGrp': ['NIA_2023_52_잔디\10_원시데이터(...,20230822133954_1267709556667_373703235833_6689...,"칼라켄터키블루그래스지역,패취병,옐로우패취,이종잔디,로이시아,크리핑벤트,화본과,바랭이_0","[{'level1': '잡초', 'level2': '화본과', 'level3': '...",0
182941,"MGC00420230822아세코벨리 1H칼라켄터키블루그래스지역,패취병,옐로우패취,이...",20230822,아세코벨리 1H,MGC004,{'originalFileGrp': ['NIA_2023_52_잔디\10_원시데이터(...,20230822133956_1267709578611_373703225833_6688...,"칼라켄터키블루그래스지역,패취병,옐로우패취,이종잔디,로이시아,크리핑벤트,화본과,바랭이_0","[{'level1': '잡초', 'level2': '화본과', 'level3': '...",0


In [37]:
newdf= newdf[newdf.numLabel ==0]
newdf.shape

(28309, 9)

In [38]:
def replaceLabel(x,y):
  x['label'] = y
  return x

newdf['json'] = newdf.apply(lambda x: replaceLabel(x.json, x.newLabel), axis = 1)

C:\Users\yc463\AppData\Local\Temp\ipykernel_34604\3455004601.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['json'] = newdf.apply(lambda x: replaceLabel(x.json, x.newLabel), axis = 1)


In [39]:
finaldf = newdf[['metadata', 'date', 'area', 'courseid', 'json', 'id', 'desc']]

In [40]:
finaldf.shape

(28309, 7)

In [41]:
finalJson = finaldf.to_dict('records')

In [42]:
finalJson[0]

{'metadata': 'MGC00520230817거창 EAST_1H미기재_1',
 'date': '20230817',
 'area': '거창 EAST_1H',
 'courseid': 'MGC005',
 'json': {'originalFileGrp': ['NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0817_거창클럽디_오전구름많이오후구름반햇빛반_데이터 1차 분류본_완\\거창 EAST_1H\\2023_0817_클럽디거창_EAST 1H_30m\\20230817_1415\\DJI_20230817141639_0046_MS_G.TIF',
   'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0817_거창클럽디_오전구름많이오후구름반햇빛반_데이터 1차 분류본_완\\거창 EAST_1H\\2023_0817_클럽디거창_EAST 1H_30m\\20230817_1415\\DJI_20230817141639_0046_MS_NIR.TIF',
   'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0817_거창클럽디_오전구름많이오후구름반햇빛반_데이터 1차 분류본_완\\거창 EAST_1H\\2023_0817_클럽디거창_EAST 1H_30m\\20230817_1415\\DJI_20230817141639_0046_MS_R.TIF',
   'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0817_거창클럽디_오전구름많이오후구름반햇빛반_데이터 1차 분류본_완\\거창 EAST_1H\\2023_0817_클럽디거창_EAST 1H_30m\\20230817_1415\\DJI_20230817141639_0046_MS_RE.TIF'],
  'annotation': [],
  'labelBy': '',
  'GeoTagInfo': {'gps_altitude': Decimal('699.912'),
   'datetime_original': '2023:08:17 14:16:39',
   'coords': [Decim

In [ ]:
with table.batch_writer(overwrite_by_pkeys=['metadata', 'id']) as batch:
  for index_, row_ in enumerate(finalJson):
    print(index_, row_['id'], end='\r')
    batch.put_item(
      Item= row_
    )